# Docker 개발 환경 구축을 위한 필수 지식

Docker는 컨테이너 기반의 가상화 플랫폼으로, 개발 환경을 일관성 있게 구성하고 배포를 단순화하는 도구입니다.

## 1. Docker 핵심 개념

### 1.1 기본 구성 요소
- **이미지(Image)**: 실행 가능한 패키지 (템플릿)
- **컨테이너(Container)**: 실행 중인 이미지 인스턴스
- **Dockerfile**: 이미지 빌드를 위한 명령어 스크립트
- **도커 데몬(Docker Daemon)**: 백그라운드 서비스
- **도커 클라이언트(Docker CLI)**: 사용자 명령 인터페이스

### 1.2 가상머신 vs 도커 컨테이너
- **가상머신**: 전체 OS 가상화 (무거움)
- **컨테이너**: 프로세스 격리 (경량, 호스트 OS 커널 공유)

## 2. 기본 설치 및 설정

### 2.1 주요 플랫폼별 설치
```bash
# Ubuntu
sudo apt-get update
sudo apt-get install docker-ce docker-ce-cli containerd.io

# Mac (Homebrew)
brew install --cask docker

# Windows: Docker Desktop 설치
```

### 2.2 설치 후 확인
```bash
docker --version
docker-compose --version
docker run hello-world
```

## 3. Dockerfile 작성 필수 지식

### 3.1 기본 구조 예제
```dockerfile
# 베이스 이미지
FROM node:16-alpine

# 작업 디렉토리 설정
WORKDIR /app

# 의존성 복사 및 설치
COPY package*.json ./
RUN npm install

# 애플리케이션 코드 복사
COPY . .

# 포트 노출
EXPOSE 3000

# 실행 명령
CMD ["npm", "start"]
```

### 3.2 주요 명령어
- `FROM`: 베이스 이미지 지정
- `RUN`: 명령어 실행 (이미지 빌드 시)
- `COPY`: 호스트 → 컨테이너 파일 복사
- `ADD`: COPY 기능 + URL/압축 파일 해제
- `CMD`: 컨테이너 실행 시 기본 명령
- `ENTRYPOINT`: CMD와 유사 but 오버라이드 어려움
- `ENV`: 환경 변수 설정
- `ARG`: 빌드 시 전달되는 변수
- `VOLUME`: 영구 데이터 저장소 마운트

## 4. 이미지 빌드 및 컨테이너 실행

### 4.1 기본 작업 흐름
```bash
# 이미지 빌드
docker build -t my-app:1.0 .

# 컨테이너 실행
docker run -d -p 4000:3000 --name my-app-container my-app:1.0

# 실행 중인 컨테이너 확인
docker ps

# 컨테이너 중지/삭제
docker stop my-app-container
docker rm my-app-container
```

### 4.2 주요 실행 옵션
- `-d`: 데몬 모드 (백그라운드 실행)
- `-p`: 포트 매핑 (호스트:컨테이너)
- `-v`: 볼륨 마운트 (호스트:컨테이너)
- `-e`: 환경 변수 설정
- `--name`: 컨테이너 이름 지정
- `--network`: 네트워크 지정

## 5. Docker Compose를 이용한 멀티 컨테이너 관리

### 5.1 docker-compose.yml 예제 (Node.js + PostgreSQL)
```yaml
version: '3.8'

services:
  web:
    build: .
    ports:
      - "3000:3000"
    environment:
      - DB_HOST=db
      - DB_PORT=5432
    depends_on:
      - db
    volumes:
      - .:/app
      - /app/node_modules

  db:
    image: postgres:13
    environment:
      POSTGRES_PASSWORD: mysecretpassword
      POSTGRES_DB: myapp
    volumes:
      - postgres_data:/var/lib/postgresql/data

volumes:
  postgres_data:
```

### 5.2 Compose 명령어
```bash
# 서비스 시작
docker-compose up -d

# 서비스 중지
docker-compose down

# 특정 서비스 로그 확인
docker-compose logs -f web

# 서비스 재빌드
docker-compose up -d --build
```

## 6. 개발 환경 최적화 기법

### 6.1 볼륨을 이용한 실시간 코드 반영
```yaml
services:
  web:
    volumes:
      - .:/app  # 호스트 코드를 컨테이너에 마운트
      - /app/node_modules  # 종속성은 별도 관리
```

### 6.2 멀티스테이지 빌드 (프로덕션 최적화)
```dockerfile
# 빌드 스테이지
FROM node:16 AS builder
WORKDIR /app
COPY . .
RUN npm install && npm run build

# 실행 스테이지
FROM node:16-alpine
WORKDIR /app
COPY --from=builder /app/dist ./dist
COPY package*.json ./
RUN npm install --production
CMD ["node", "dist/main.js"]
```

### 6.3 .dockerignore 파일
```
node_modules
.git
.env
*.log
dist
```

## 7. 네트워킹 기본

### 7.1 기본 네트워크 타입
- **bridge**: 기본 네트워크 (동일 호스트 내 컨테이너 통신)
- **host**: 호스트 네트워크 직접 사용
- **overlay**: 여러 도커 데몬 간 통신 (Swarm 모드)

### 7.2 사용자 정의 네트워크 생성
```bash
docker network create my-network
docker run --network=my-network my-app
```

## 8. 개발 워크플로우 예시

1. 애플리케이션 코드 작성
2. Dockerfile 정의
3. 필요한 서비스 docker-compose.yml에 구성
4. 개발 시:
   ```bash
   docker-compose up -d
   ```
5. 코드 변경 시 자동 반영(볼륨 마운트)
6. 테스트 완료 후 이미지 빌드:
   ```bash
   docker build -t my-app:1.0 .
   ```
7. 레지스트리에 푸시:
   ```bash
   docker tag my-app:1.0 myrepo/my-app:1.0
   docker push myrepo/my-app:1.0
   ```

## 9. 고급 개발 팁

### 9.1 디버깅
```bash
# 실행 중인 컨테이너에 접속
docker exec -it my-container /bin/bash

# 로그 확인
docker logs -f my-container
```

### 9.2 개발용 vs 프로덕션용 구성
```yaml
# docker-compose.yml
services:
  web:
    build:
      context: .
      target: development  # Dockerfile에서 ARG 사용

# docker-compose.prod.yml
services:
  web:
    build:
      context: .
      target: production
```

### 9.3 Health Check
```dockerfile
HEALTHCHECK --interval=30s --timeout=3s \
  CMD curl -f http://localhost:3000/health || exit 1
```

## 10. 보안 모범 사례

1. **최소 권한 원칙**: root 계정 사용 지양
   ```dockerfile
   USER node
   ```
2. **이미지 경량화**: alpine 기반 이미지 사용
3. **비밀값 관리**: Docker Secrets 또는 환경 변수
4. **정기 업데이트**: 베이스 이미지 주기적 업데이트
5. **스캐닝 도구**: Clair, Trivy로 취약점 검사

Docker 개발 환경 구축은 "코드로서의 인프라" 접근 방식을 가능하게 하여, 모든 팀원이 동일한 환경에서 작업할 수 있도록 합니다. 점진적으로 네트워킹, 볼륨, 오케스트레이션 등의 고급 개념을 익히는 것을 추천합니다.